In [6]:
import pyspark 
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *


spark=SparkSession.builder.master("yarn")\
        .appName("movie")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark-3.5.3/jars/mysql-connector-j-9.1.0.jar")\
        .config("spark.sql.shuffle.partitions", "7") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.executor.cores", "2") \
        .config("spark.dynamicAllocation.minExecutors", "1") \
        .config("spark.dynamicAllocation.maxExecutors", "5") \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/06 21:12:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/06 21:12:33 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/01/06 21:12:34 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
25/01/06 21:12:48 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/01/06 21:12:48 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


PySpark Version :3.5.3
PySpark Version :3.5.3


In [2]:
df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
df.show()

+---------+-------------------+---------+---------+--------------------+--------------------+
|   nconst|        primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+-------------------+---------+---------+--------------------+--------------------+
|nm0000001|       Fred Astaire|     1899|     1987|soundtrack,actor,...|tt0050419,tt00723...|
|nm0000002|      Lauren Bacall|     1924|     2014|  actress,soundtrack|tt0037382,tt00383...|
|nm0000003|    Brigitte Bardot|     1934|       \N|actress,soundtrac...|tt0056404,tt00573...|
|nm0000004|       John Belushi|     1949|     1982|actor,soundtrack,...|tt0077975,tt00725...|
|nm0000005|     Ingmar Bergman|     1918|     2007|writer,director,a...|tt0083922,tt00509...|
|nm0000006|     Ingrid Bergman|     1915|     1982|actress,soundtrac...|tt0077711,tt00345...|
|nm0000007|    Humphrey Bogart|     1899|     1957|actor,soundtrack,...|tt0043265,tt00345...|
|nm0000008|      Marlon Brando|     1924|     2004|actor,sou

In [6]:
df.show(truncate=False)


+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nconst   |primaryName        |birthYear|deathYear|primaryProfession                    |knownForTitles                         |
+---------+-------------------+---------+---------+-------------------------------------+---------------------------------------+
|nm0000001|Fred Astaire       |1899     |1987     |soundtrack,actor,miscellaneous       |tt0050419,tt0072308,tt0031983,tt0053137|
|nm0000002|Lauren Bacall      |1924     |2014     |actress,soundtrack                   |tt0037382,tt0038355,tt0117057,tt0071877|
|nm0000003|Brigitte Bardot    |1934     |\N       |actress,soundtrack,music_department  |tt0056404,tt0057345,tt0049189,tt0054452|
|nm0000004|John Belushi       |1949     |1982     |actor,soundtrack,writer              |tt0077975,tt0072562,tt0080455,tt0078723|
|nm0000005|Ingmar Bergman     |1918     |2007     |writer,director,actor                |t

In [21]:
# df.filter("primaryProfession in ('soundtrack')").show()
df = (df.withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
        .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))

)
dfs = df.select(F.col("nconst"),
                F.col("primaryName"),
                F.col("birthYear"),
                F.col("deathYear"),
                F.explode(F.col("profession_array")).alias("profession"),
                F.explode(F.col("knownForTitles_array")).alias("Titles")).show()
                



+---------+-------------+---------+---------+-------------+---------+
|   nconst|  primaryName|birthYear|deathYear|   profession|   Titles|
+---------+-------------+---------+---------+-------------+---------+
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0031983|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0053137|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0031983|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0053137|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0031983|
|nm0000001| Fred Ast

In [7]:
from pyspark.sql import SparkSession,Row,DataFrame

def dp_etl(df: DataFrame) -> DataFrame :

    dffull = (df.withColumn('profession_array',(F.split(F.col("primaryProfession"), ",")))
            .withColumn('knownForTitles_array',(F.split(F.col("knownForTitles"),",")))
            .select(F.col("nconst"),
                     F.col("primaryName"),
                     F.col("birthYear"),
                     F.col("deathYear"),
                     F.explode(F.col("profession_array")).alias("profession"),
                     F.explode(F.col("knownForTitles_array")).alias("Titles")))
    return dffull


df = spark.read.option("delimiter", "\t").option("header", True).csv('hdfs:/user/bigdata/repository/input/data_2022*.csv')
dp_etl(df).show()

    

+---------+-------------+---------+---------+-------------+---------+
|   nconst|  primaryName|birthYear|deathYear|   profession|   Titles|
+---------+-------------+---------+---------+-------------+---------+
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0031983|
|nm0000001| Fred Astaire|     1899|     1987|   soundtrack|tt0053137|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0031983|
|nm0000001| Fred Astaire|     1899|     1987|        actor|tt0053137|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0050419|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0072308|
|nm0000001| Fred Astaire|     1899|     1987|miscellaneous|tt0031983|
|nm0000001| Fred Ast